In [2]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import string
from wordcloud import WordCloud
from textblob import TextBlob
import seaborn as sns

In [2]:
https://towardsdatascience.com/text-clustering-using-k-means-ec19768aae48

SyntaxError: invalid syntax (Temp/ipykernel_14016/3313368796.py, line 1)

In [1]:
def read_nGrams():
    """
    This function will read bigrams & trigrams and 
    return combined list of bigrams & trigrams.
    """
    # read  bigrams 
    original_bigram = readFile("bigram.txt")
    # read trigrams
    original_trigram = readFile("trigram.txt")

    # Combined list of bigrams & trigrams
    n_grams_to_use = []
    n_grams_to_use.extend(original_bigram)
    n_grams_to_use.extend(original_trigram)
    return n_grams_to_use
n_grams_to_use = read_nGrams()

NameError: name 'readFile' is not defined